# CS 524 - Homework 9

## Devin Johnson (djohnson58)

### Problem 1

In [9]:
using JuMP, Gurobi

# Number of districts desired
d = 5

# Number of cities
c = 10

# Make a 2d array of cities and their respective dems/repubs
data =    [80 34
           60 44
           40 44
           20 24
           40 114
           40 64
           70 14
           50 44
           70 54
           70 64]

m = Model(solver=GurobiSolver(OutputFlag=0))

# Let us know whether a district has a majority of democrats (1 if yes, 0 if no)
@variable(m, dem_maj[1:d], Bin)

# Let us know if a city belongs to a district (1 if yes, 0 if no)
@variable(m, district_assignment[1:c, 1:d], Bin)


# For each district
for i = 1:d
    # 150 <= City's (repubs + dems) * (assignment to this district 0 or 1) <= 250
    # make sure voter limits not surpassed
    @constraint(m, 150 <= sum((data[j,1] + data[j,2])*district_assignment[j,i] 
            for j = 1:c))
end

# For each district
for i = 1:d
    # 150 <= City's (repubs + dems) * (assignment to this district 0 or 1) <= 250
    # make sure voter limits not surpassed
    @constraint(m, sum((data[j,1] + data[j,2])*district_assignment[j,i] 
            for j = 1:c) <= 250)
end

# For each district
for i = 1:d
    @constraint(m, sum((data[j,2] - data[j,1])*district_assignment[j,i] 
            for j = 1:c) >= -250 *(1-dem_maj[i]))
end

# Each city should only belong to one district
for i = 1:c
    @constraint(m, sum(district_assignment[i,:]) == 1)
end

# Get the most democratic majority districts
@objective(m, Max, sum(dem_maj))

status = solve(m)

println(status)
println("Districts with majority: ", getobjectivevalue(m))
println("District assignments: ", getvalue(district_assignment))

Academic license - for non-commercial use only
Optimal
Districts with majority: 3.0
District assignments: [0.0 0.0 0.0 1.0 0.0; 0.0 1.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 1.0 0.0 0.0]


### Problem 2

In [67]:
# Function to print board like from sudoku example
function print_board(arr)
    println("*---*---*---*---*---*---*---*---*")
    for i = 1:8
        for j = 1:8
            print("|")
            if(arr[i,j] == 1)
                print(" x ")
            else
                print("   ")
            end
        end
        println("|")
        println("*---*---*---*---*---*---*---*---*")
    end
end

print_board (generic function with 1 method)

#### Part A

In [68]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# Make the 8x8 board
@variable(m, board[1:8, 1:8], Bin)

# Each row must sum to one (no queens per same row)
for i = 1:8
   @constraint(m, sum(board[i,j] for j = 1:8) == 1)
end

# Each column must sum to one (no queens per same column)
for i = 1:8
   @constraint(m, sum(board[j,i] for j = 1:8) == 1)
end

# We can divide work based on the two main diagonals
# Left side of (bottom left to top right) diagonal 
for col = 2:9
    @constraint(m, sum(board[row,col-row] for row = 1:col-1) <= 1)
end

# Right side of (bottom left to top right) diagonal 
for col = 10:16
    @constraint(m, sum(board[row,col-row] for row = col-8:8) <= 1)
end

# Left side of (top left to bottom right) diagonal
for col = -7:-1
    @constraint(m, sum(board[row, row+col] for row = -col+1:8) <= 1)
end

# Right side of (top left to bottom right) diagonal
for col = 0:7
    @constraint(m, sum(board[row, row+col] for row = 1:8-col) <= 1)
end

# Minimum assigned
@objective(m, Min, sum(board))

status = solve(m)
println(status)
print_board(getvalue(board))

Academic license - for non-commercial use only
Optimal
*---*---*---*---*---*---*---*---*
|   |   |   | x |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   | x |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   | x |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   | x |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
| x |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   | x |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   | x |
*---*---*---*---*---*---*---*---*
|   | x |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*


#### Part B

In [69]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# Make the 8x8 board
@variable(m, board[1:8, 1:8], Bin)

# Each row must sum to one (no queens per same row)
for i = 1:8
   @constraint(m, sum(board[i,j] for j = 1:8) == 1)
end

# Each column must sum to one (no queens per same column)
for i = 1:8
   @constraint(m, sum(board[j,i] for j = 1:8) == 1)
end

# We can divide work based on the two main diagonals
# Left side of (bottom left to top right) diagonal 
for col = 2:9
    @constraint(m, sum(board[row,col-row] for row = 1:col-1) <= 1)
end

# Right side of (bottom left to top right) diagonal 
for col = 10:16
    @constraint(m, sum(board[row,col-row] for row = col-8:8) <= 1)
end

# Left side of (top left to bottom right) diagonal
for col = -7:-1
    @constraint(m, sum(board[row, row+col] for row = -col+1:8) <= 1)
end

# Right side of (top left to bottom right) diagonal
for col = 0:7
    @constraint(m, sum(board[row, row+col] for row = 1:8-col) <= 1)
end

# Point symmetry
for row = 1:8
    for col = 1:8
        @constraint(m, board[row,col] == board[9-row, 9-col])
    end
end

# Minimum assigned
@objective(m, Min, sum(board))

status = solve(m)
println(status)
print_board(getvalue(board))

Academic license - for non-commercial use only
Optimal
*---*---*---*---*---*---*---*---*
|   |   |   |   |   | x |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   | x |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   | x |   |
*---*---*---*---*---*---*---*---*
| x |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   | x |
*---*---*---*---*---*---*---*---*
|   | x |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   | x |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   | x |   |   |   |   |   |
*---*---*---*---*---*---*---*---*


#### Part C

In [70]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# Make the 8x8 board
@variable(m, board[1:8, 1:8], Bin)

# Each cell needs to have a queen in its same column, row, or diagonal
for row = 1:8
    for col = 1:8
        @constraint(m, (sum(board[row,d] for d = 1:8) + 
                        sum(board[d,col] for d = 1:8) +
                        sum(board[d,row+col-d] for d = max(1,(row+col-8)):min(8,(row+col-1))) +
                        sum(board[d,col-row+d] for d = max(1,-col+row+1):min(8,8-col+row)))
                        >= 1)
    end
end

@objective(m, Min,sum(board))
status = solve(m)
println(status)
print_board(getvalue(board))

Academic license - for non-commercial use only
Optimal
*---*---*---*---*---*---*---*---*
|   |   | x |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   | x |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   | x |   |   |   |
*---*---*---*---*---*---*---*---*
| x |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   | x |   |   |
*---*---*---*---*---*---*---*---*


#### Part D

In [71]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

# Make the 8x8 board
@variable(m, board[1:8, 1:8], Bin)

# Each cell needs to have a queen in its same column, row, or diagonal
for row = 1:8
    for col = 1:8
        @constraint(m, (sum(board[row,d] for d = 1:8) + 
                        sum(board[d,col] for d = 1:8) +
                        sum(board[d,row+col-d] for d = max(1,(row+col-8)):
                        min(8,(row+col-1))) +
                        sum(board[d,col-row+d] for d = max(1,-col+row+1):
                        min(8,8-col+row)))
                        >= 1)
    end
end

# Point symmetry
for row = 1:8
    for col = 1:8
        @constraint(m, board[row,col] == board[9-row, 9-col])
    end
end

@objective(m, Min,sum(board))
status = solve(m)
println(status)
print_board(getvalue(board))

Academic license - for non-commercial use only
Optimal
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   | x |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   | x |
*---*---*---*---*---*---*---*---*
|   |   |   | x |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   | x |   |   |   |
*---*---*---*---*---*---*---*---*
| x |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   | x |   |   |
*---*---*---*---*---*---*---*---*
|   |   |   |   |   |   |   |   |
*---*---*---*---*---*---*---*---*


Yes, the number changes. Now we have 6 queens.

### Problem 3

In [80]:
using JuMP, Gurobi

# Average run times for 400 meters
# I'm assuming this is just something we can add
# onto the objective since everyone has to run anyway
avg_time = [82.5 77.1 81.3 74.9 80.6]

# Take over times
take_over =    [0 1.1 1.3 1.9 2.1
               1.2 0 1.7 1.0 1.8
               1.7 1.4 0 0.9 1.7
               2.1 0.8 1.6 0 2.4
               1.5 1.2 1.9 2.3 0]

m = Model(solver=GurobiSolver(OutputFlag=0))

# Decide which relay to do
@variable(m, x[1:5,1:5], Bin)

# It doesn't make sense for a runner to pass to themself
for row = 1:5
    @constraint(m, x[row,row] == 0)
end

# Must be 4 passes
@constraint(m, sum(x) == 4)

# Sum of rows <= 1
for i = 1:5
   @constraint(m, sum(x[i,j] for j = 1:5) <=  1)
end

# Sum of cols <= 1
for i = 1:5
   @constraint(m, sum(x[j,i] for j = 1:5) <= 1)
end

# Every row + its corresponding column sums to at least 1
# 1 if only sending or receiving and 2 if sending and receiving
for i = 1:5
    @constraint(m, 1 <= sum(x[i,j] for j = 1:5) + sum(x[j,i] for j = 1:5))
end
for i = 1:5
    @constraint(m, sum(x[i,j] for j = 1:5) + sum(x[j,i] for j = 1:5) <= 2)
end

# Times every cell by take over time
# Objective is to minmize the takeover time for 5 runners
@objective(m, Min, sum(x[i,j]*take_over[i,j] for i = 1:5, j = 1:5))

status = solve(m)
println("Run time: ", getobjectivevalue(m) + sum(avg_time))
println("Assignment: ")
println("     A | B | C | D | E |")
for i = 1:5
    if i == 1
        print("A|  ")
    end
    
    if i == 2
        print("B|  ")
    end
    
    if i == 3
        print("C|  ")
    end
    
    if i == 4
        print("D|  ")
    end
    
    if i == 5
        print("E|  ")
    end
    
    for j = 1:5
        if j == 5
            println(abs(getvalue(x[i,j])))
        else
            print(abs(getvalue(x[i,j])), " ")
        end
    end
end


Academic license - for non-commercial use only
Run time: 400.9
Assignment: 
     A | B | C | D | E |
A|  0.0 0.0 1.0 0.0 0.0
B|  0.0 0.0 0.0 0.0 0.0
C|  0.0 0.0 0.0 1.0 0.0
D|  0.0 1.0 0.0 0.0 0.0
E|  1.0 0.0 0.0 0.0 0.0


So the order is Elisa, Alice, Cindy, David, Bob